In [56]:
import pandas as pd
import numpy as np

from functools import reduce

In [57]:
# factorial of digits
fact = [1]
for i in range(1,10):
    fact.append(fact[-1]*i)

dfs_cache = np.zeros(6*fact[9] + 1, int)
def digit_fact_sum(n):
    if dfs_cache[n] == 0:
        dfs_cache[n] = reduce(lambda a,b: a + fact[int(b)], str(n), 0)
    
    return dfs_cache[n]

At first I solved this problem by simply looping through every number, and finding its chain length, just to see if it was practically possible (using caching on the factorials and the digit-factorial-sums, for slight optimizations). It actually solved the problem in ~40s. 

But we can do better by realizing once we see a number, we are already seeing it's chain in action; so if we ever encounter it again, we already know what it's chain length will be! In particular, if we have seen some number before, we can break the loop/recursion and just add that value onto the end of the loop/recursion.

Here is a recursion version of the solution, which works like most cached recursion solutions. The function $LOC(x)$, which represents the length of chain of $x \in \mathbb{N}$, works as follows:

0. Initialize a cache $c(x) = 0$ for every $x \in \mathbb{N}$ that is of interest. For this problem, I used $x < 3 \cdot 10^6$. Also, pre-cache all the digit factorials from $0!$ to $9!$, and create a function $\text{DFS}(x)$ that does the digit-factorial-sum of a number $x \in \mathbb{N}$ (this is also cached for optimization). 
1. We are given a value $n$. If we have seen it already in this chain, then the chain has repeated, so return $0$.
2. If you have seen $n$ in a previous chain, but not before in this current chain (i.e., $n$ is already in the cache $c(n)$, so $c(n) > 0$), then you can just return the cached value, $c(n)$.
3. If you have not seen $n$ before (i.e., $c(n) = 0$), then we need to populate the chain. We add $n$ to the chain and apply the recursion to the next value, $\text{DFS}(n)$. We add $1$ to the result of the chain on the next value, so we set the cached value to $c(n) = 1 + \text{LengthOfChain}(DFS(n))$.
4. Return the cached value, $c(n)$.

In [68]:
chain_cache = np.zeros(3*10**6, int)
def length_of_chain(n, seen_so_far = set()):
    if n in seen_so_far:
        return 0

    if chain_cache[n] == 0:
        seen = []
        curr = n

        new_set = seen_so_far.copy()
        new_set.add(n)
        chain_cache[n] = 1 + length_of_chain(digit_fact_sum(curr), new_set)

    return chain_cache[n]

max_chain = 0
for i in range(10**6)[::-1]:
    max_chain += 1 if length_of_chain(i) >= 60 else 0

print(max_chain)

402


Here is an iterative solution, which works similarly with caching. Here we increment the cache based on the list of elements we have seen so far. But more or less, the logic matches what came in the recursion solution.

In [69]:
max_chain = 0
seen_cache = np.zeros(3*10**6, int)
for n in range(10**6)[::-1]:    
    if seen_cache[n] >= 60:
        max_chain += 1
        continue

    seen = []
    curr = n

    go_to_cache = False
    while curr not in seen:
        seen.append(curr)
        curr = digit_fact_sum(curr)
        if seen_cache[curr]:
            go_to_cache = True
            break
    
    if go_to_cache:
        nums_seen = len(seen) + seen_cache[curr]
    else:
        nums_seen = len(seen)

    ind = None
    for i in range(len(seen)):
        if seen[i] == curr:
            ind = i

        seen_cache[seen[i]] = nums_seen - ind if ind else nums_seen - i

    max_chain += 1 if seen_cache[n] >= 60 else 0

print(max_chain)

402
